<a href="https://colab.research.google.com/github/sandrakaku/ml0930/blob/master/titanic_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/sandrakaku/ml0930/raw/master/titanic/train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/sandrakaku/ml0930/raw/master/titanic/test.csv"
urlretrieve(url, "test.csv")

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [ ]:
datas = pd.concat([train_df, test_df], axis=0, ignore_index=True)

In [ ]:
datas = datas.drop(["PassengerId", "Survived"], axis=1) #設10個

In [ ]:
na = datas.isna().sum() #.isna()檢查裡面是不是空的(空的為True) .sum()True+1
# 篩選 Series[True/False list/series]
na[na > 0].sort_values(ascending=False) #有缺就會是True +上[]會篩選出True的 /sort_values(升序(ascending)|降序(decending))

In [ ]:
most = datas["Embarked"].value_counts().idxmax() #.idxmax找最大值的index
datas["Embarked"] = datas["Embarked"].fillna(most) #把剩下空的都填最大值index進去 (如果是數值基本上取中位數去補缺失)

In [ ]:
# 先處理一下ticket
dic = datas["Ticket"].value_counts() #多少人擁有同一張票
def count(n):
  return dic[n]
datas["Ticket"] = datas["Ticket"].apply(count)

In [ ]:
med = datas.median().drop(["Pclass"])
datas = datas.fillna(med)

In [ ]:
# 一個段落，看一下還缺啥
na = datas.isna().sum() #.isna()檢查裡面是不是空的(空的為True) .sum()True+1
# 篩選 Series[True/False list/series]
na[na > 0].sort_values(ascending=False) #有缺就會是True +上[]會篩選出True的 /sort_values(升序(ascending)|降序(decending))

In [ ]:
# 處理Cabin
def cabin(c):
  if pd.isna(c):
    return None
  else:
    return c[0]
datas["Cabin"] = datas["Cabin"].apply(cabin) #Cabin空的補None 有東西取第一個號碼

In [ ]:
def name(n):
  mid = n.split(",")[-1].split(".")[0] #將Name的"，"當分界分開字串取倒數第一個,再用"."當分界取第一個
  mid = mid.strip() #將字前後多餘的空白鍵去掉
  return mid
middle = datas["Name"].apply(name)
counts = middle.value_counts()
reserved = counts[:4].index
def name2(mid):
  if mid in reserved:
    return mid
  else:
    return None
datas["Name"] = middle.apply(name2)

In [ ]:
datas = pd.get_dummies(datas) #所有文字做One-Hot encoding出來
datas = pd.get_dummies(datas, columns=["Pclass"])

In [ ]:
datas["Family"] = datas["SibSp"] + datas["Parch"]
datas

In [ ]:
# [first row, second row....] 類似List去取要東西 DataFrame
x_train = datas.iloc[:len(train_df)]
x_predict = datas.iloc[len(train_df):]
y_train = train_df["Survived"]
x_predict

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
params = {
    "n_estimators":range(20, 21),
    "max_depth":range(4, 5)
}
clf = RandomForestClassifier(random_state=1)
cv = GridSearchCV(clf, params, cv=10, n_jobs=-1)
cv.fit(x_train, y_train)
print(cv.best_score_)
print(cv.best_params_)

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=63, max_depth=7) #如果沒有寫random_state=1，每次取平均值的數都會不一樣
scores = cross_val_score(clf, x_train, y_train, cv=10, n_jobs=-1)
print(scores)
print(np.average(scores)) #隨機性是來自隨機森林(他會隨機放掉一些資料)

In [ ]:
clf = RandomForestClassifier(n_estimators=63, max_depth=7)
clf.fit(x_train, y_train)
pre = clf.predict(x_predict)
df = pd.DataFrame({
    "PassengerId":test_df["PassengerId"],
    "Survived":pre
})
df.to_csv("rf.csv", encoding="utf-8", index=False)

In [ ]:
clf.estimators_[0]
pd.DataFrame({
    "columns":x_train.columns,
    "importance":clf.feature_importances_
}).sort_values("importance", axis=0, ascending=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10, 4)) 
plt.subplot(1, 2, 1) #畫圖前
sns.countplot(train_df["Sex"])
plt.subplot(1, 2, 2)
sns.countplot(x=train_df["Sex"], hue=train_df["Survived"], palette="Paired") #palette換顏色

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(2, 1,1)
sns.distplot(train_df["Age"])
c = pd.cut(train_df["Age"], bins=5)
plt.subplot(2, 1, 2)
sns.countplot(x=c, hue=train_df["Survived"])
plt.xticks(rotation=45) #讓x軸的區塊名稱傾斜

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
datas_scale = scaler.fit_transform(datas)
datas_scale = pd.DataFrame(datas_scale, columns=datas.columns)
x_train_scale = datas_scale.iloc[:len(train_df)]
x_predict_scale = datas_scale.iloc[len(train_df):]
y_train_scale = train_df["Survived"]
x_predict

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
params = {
    "n_neighbors":range(5, 100),
}
clf = KNeighborsClassifier()
cv = GridSearchCV(clf, params, cv=10, n_jobs=-1)
cv.fit(x_train_scale, y_train)
print(cv.best_score_)
print(cv.best_params_)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=11)
clf.fit(x_train_scale, y_train)
pre = clf.predict(x_predict_scale)
df = pd.DataFrame({
    "PassengerId":test_df["PassengerId"],
    "Survived":pre
})
df.to_csv("knn.csv", encoding="utf-8", index=False)